In [1]:
import numpy as np
import pickle
import pandas as pd

import keras.backend as K
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Masking, CuDNNLSTM, TimeDistributed, \
                         Bidirectional, Flatten, \
                         Embedding, Dropout, Flatten, BatchNormalization, \
                         RNN, concatenate, Activation, LSTM
from keras.callbacks import EarlyStopping
from keras.preprocessing import sequence
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from keras.layers.convolutional import Conv2D
import random as rn
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
import codecs
from tensorflow.contrib.rnn import *

Using TensorFlow backend.


In [2]:
rn.seed(123)
np.random.seed(99)
tf.set_random_seed(1234)

In [3]:
# load feature and labels
feat = np.load('../../dimensionalSM/data/feat_34_hfs.npy')
vad = np.load('../../dimensionalSM/data/y_egemaps.npy')
# remove outlier, < 1, > 5
vad = np.where(vad==5.5, 5.0, vad)
vad = np.where(vad==0.5, 1.0, vad)

# standardization
scaled_feature = False

In [4]:
#path = '/home/s1820002/IEMOCAP-Emotion-Detection/'
data = np.load('../../dimensionalSM/IEMOCAP_full_release/data_collected_10039.pickle', allow_pickle=True)
text = [t['transcription'] for t in data]
#vad = np.load('../../data/y_egemaps.npy')
# load fastext model
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text)

token_tr_X = tokenizer.texts_to_sequences(text)
MAX_SEQUENCE_LENGTH = len(max(token_tr_X, key=len))
x_train_text = []
x_train_text = sequence.pad_sequences(token_tr_X, maxlen=MAX_SEQUENCE_LENGTH)

word_index = tokenizer.word_index
nb_words = len(word_index) +1
print('Found %s unique tokens' % len(word_index))

# load model
EMBEDDING_DIM = 300
file_loc = '../../dimensionalSM/data/glove.42B.300d.txt'
print (file_loc)

gembeddings_index = {}
with codecs.open(file_loc, encoding='utf-8') as f:
    for line in f:
        values = line.split(' ')
        word = values[0]
        gembedding = np.asarray(values[1:], dtype='float32')
        gembeddings_index[word] = gembedding
#
f.close()
print('G Word embeddings:', len(gembeddings_index))

nb_words = len(word_index) +1
g_word_embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
for word, i in word_index.items():
    gembedding_vector = gembeddings_index.get(word)
    if gembedding_vector is not None:
        g_word_embedding_matrix[i] = gembedding_vector
        
print('G Null word embeddings: %d' % np.sum(np.sum(g_word_embedding_matrix, axis=1) == 0))

# split train/test
split = 7869
earlystop = EarlyStopping(monitor='val_loss', patience=10, mode='min', 
                          restore_best_weights=True)


Found 3437 unique tokens
../../dimensionalSM/data/glove.42B.300d.txt
G Word embeddings: 1917494
G Null word embeddings: 128


In [5]:
import pickle
with open('../../dimensionalSM/IEMOCAP_full_release/data_collected_333.pickle', 'rb') as handle:
    data2 = pickle.load(handle)
x_train_mocap = []
counter = 0
for ses_mod in data2:
    x_head = ses_mod['mocap_head']
    if(x_head.shape != (200,18)):
        x_head = np.zeros((200,18))   
    x_head[np.isnan(x_head)]=0
    x_hand = ses_mod['mocap_hand']
    if(x_hand.shape != (200,6)):
        x_hand = np.zeros((200,6))   
    x_hand[np.isnan(x_hand)]=0
    x_rot = ses_mod['mocap_rot']
    if(x_rot.shape != (200,165)):
        x_rot = np.zeros((200,165))   
    x_rot[np.isnan(x_rot)]=0
    x_mocap = np.concatenate((x_head, x_hand), axis=1)
    x_mocap = np.concatenate((x_mocap, x_rot), axis=1)
    x_train_mocap.append( x_mocap )
    
x_train_mocap = np.array(x_train_mocap)
x_train_mocap = x_train_mocap.reshape(-1,200,189,1)
x_train_mocap.shape

(10039, 200, 189, 1)

In [6]:
if scaled_feature == True:
    scaler = StandardScaler()
    scaler = scaler.fit(feat.reshape(feat.shape[0]*feat.shape[1], feat.shape[2]))
    scaled_feat = scaler.transform(feat.reshape(feat.shape[0]*feat.shape[1], feat.shape[2]))
    scaled_feat = scaled_feat.reshape(feat.shape[0], feat.shape[1], feat.shape[2])
    feat = scaled_feat
else:
    feat = feat

scaled_vad = True

# standardization
if scaled_vad:
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler = scaler.fit(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    scaled_vad = scaler.transform(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    vad = scaled_vad 
else:
    vad = vad

# Concordance correlation coefficient (CCC)-based loss function - using non-inductive statistics
def ccc(gold, pred):
    gold       = K.squeeze(gold, axis=-1)
    pred       = K.squeeze(pred, axis=-1)
    gold_mean  = K.mean(gold, axis=-1, keepdims=True)
    pred_mean  = K.mean(pred, axis=-1, keepdims=True)
    covariance = (gold-gold_mean)*(pred-pred_mean)
    gold_var   = K.mean(K.square(gold-gold_mean), axis=-1, keepdims=True)
    pred_var   = K.mean(K.square(pred-pred_mean), axis=-1, keepdims=True)
    ccc        = K.constant(2.) * covariance / (gold_var + pred_var + K.square(gold_mean - pred_mean) + K.common.epsilon())
    return ccc


def ccc_loss(gold, pred):  
    # input (num_batches, seq_len, 1)
    ccc_loss   = K.constant(1.) - ccc(gold, pred)
    return ccc_loss


In [7]:
# API model
def model():
    # speech network
#     input_speech = Input(shape=(feat.shape[1], feat.shape[2]), name='speech_input')
#     net_speech = BatchNormalization()(input_speech)
#     net_speech = LSTM(32, return_sequences=True)(net_speech)
#     net_speech = LSTM(64, return_sequences=True)(net_speech)
#     net_speech = LSTM(128, return_sequences=True)(net_speech)
#     net_speech = LSTM(64, return_sequences=True)(net_speech)
#     net_speech = LSTM(32, return_sequences=True)(net_speech)
#     model_speech = Flatten()(net_speech)
    #model_speech = Dropout(0.1)(net_speech)
    
    #text network
    input_text = Input(shape=(MAX_SEQUENCE_LENGTH, ))
    net_text = Embedding(nb_words,
                    EMBEDDING_DIM,
                    weights = [g_word_embedding_matrix],
                    trainable = True)(input_text)
    net = Bidirectional(LSTM(128, return_sequences=True))(net_text)
    net = Bidirectional(LSTM(64, return_sequences=False))(net)  
    model_text = Dense(64)(net)
    
    #mocap network
    input_mocap = Input(shape=(200,189,1))
    model_mocap0 = BatchNormalization()(input_mocap)
    model_mocap1 = Conv2D(64, 3, strides=(3, 3), padding="same")(model_mocap0)
    model_mocap2 = Dropout(0.2)(model_mocap1)
    model_mocap3 = Activation('relu')(model_mocap2)
    model_mocap4 = Conv2D(128, 3,strides=(3, 3), padding="same")(model_mocap3)
    model_mocap5 = Dropout(0.2)(model_mocap4)
    model_mocap6 = Activation('relu')(model_mocap5)
    model_mocap7 = Conv2D(256, 3,strides=(3, 3), padding="same")(model_mocap6)
    model_mocap8 = Dropout(0.2)(model_mocap7) 
    model_mocap9 = Activation('relu')(model_mocap8)
    model_mocap10 = Conv2D(128, 3,strides=(3, 3), padding="same")(model_mocap9)   
    model_mocap11 = Dropout(0.2)(model_mocap10)  
    model_mocap12 = Activation('relu')(model_mocap11)
    model_mocap13 = Conv2D(64, 3,strides=(3, 3), padding="same")(model_mocap12)  
    model_mocap14 = Dropout(0.2)(model_mocap13) 
    model_mocap15 = Activation('relu')(model_mocap14)
    model_mocap = Flatten()(model_mocap15) 
    #model_mocap = Dropout(dropout_rate)(model_mocap16)
    # combined model
    model_combined = concatenate([model_text, model_mocap])
    model_combined = Dense(64, activation='relu')(model_combined)
    model_combined = Dense(32, activation='relu')(model_combined)
    #model_combined = Dropout(0.4)(model_combined)
    target_names = ('v', 'a', 'd')
    model_combined = [Dense(1, name=name)(model_combined) for name in target_names]

    model = Model([input_text, input_mocap], model_combined) 
    model.compile(loss=ccc_loss,
                  loss_weights={'v': 1, 'a': 1, 'd': 1},
                  optimizer='rmsprop', metrics=[ccc])
    return model


# model_1 = model()
# model_1.summary()

In [8]:
# #speech_to_npy
# model1 = model()
# earlystop = EarlyStopping(monitor='val_loss', mode='min', patience=10,
#                               restore_best_weights=True)
# hist1 = model1.fit([x_train_text[:8000],x_train_mocap[:8000]], vad[:8000].T.tolist(), epochs=50, 
#                    batch_size=32, verbose=1, validation_split=0.2, 
#                    callbacks=[earlystop])

# eval_metrik1 = model1.evaluate([x_train_text[8000:],x_train_mocap[8000:]], vad[8000:].T.tolist())
# print(eval_metrik1)

# # make prediction
# predict = model1.predict([x_train_text[6400:],x_train_mocap[6400:]], batch_size=32)
# np.save('text_mocapGLOVE_npy8000', 
#          np.array(predict).reshape(3, 3639).T)


In [9]:
# main function for for LOSO
def main(time):
    model_1 = model()
    earlystop = EarlyStopping(monitor='val_loss', mode='min', patience=10,
                              restore_best_weights=True)
    hist = model_1.fit( [x_train_text[:8000],x_train_mocap[:8000]],
                      vad[:8000].T.tolist(), batch_size=32, #best:8
                      validation_split=0.2, epochs=50, verbose=1, shuffle=True,
                      callbacks=[earlystop])
    metrik = model_1.evaluate( [x_train_text[8000:],x_train_mocap[8000:]], vad[8000:].T.tolist())
    print(metrik[-3:])
    # make prediction
    predict = model_1.predict([x_train_text[6400:],x_train_mocap[6400:]], batch_size=32)
    np.save('npy8000/text_mocap_npy-8000batch'+str(time)+'.npy',  
             np.array(predict).reshape(3, 3639).T)
    return metrik
    
if __name__ == '__main__':
    #loss_weight_step = np.around(np.arange(0.0, 1.1, 0.1), decimals=1)
    #dropout_rates = np.arange(0.5, 0.8, 0.1)
    weight = []
    #for i in dropout_rates:
    for time in range(20):
        best = main(time)
        weight.append([time, best[-3], best[-2], best[-1], np.mean(best[-3:])])
        print(time)
    weight_df = pd.DataFrame(weight)
    weight_df.to_csv('npy8000/tm_glove_8000_batch-1.csv', header=['time', 'v','a','d', 'ave'])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Train on 6400 samples, validate on 1600 samples
Epoch 1/50
6400/6400 [==============================] - 92s 14ms/step - loss: 1.9790 - v_loss: 0.6626 - a_loss: 0.6586 - d_loss: 0.6578 - v_ccc: 0.3374 - a_ccc: 0.3414 - d_ccc: 0.3422 - val_loss: 2.1091 - val_v_loss: 0.7429 - val_a_loss: 0.6740 - val_d_loss: 0.6922 - val_v_ccc: 0.2571 - val_a_ccc: 0.3260 - val_d_ccc: 0.3078
Epoch 2/50
6400/6400 [==============================] - 87s 14ms/step - loss: 1.4846 - v_loss: 0.4079 - a_loss: 0.5386 - d_loss: 0.5381 - v_ccc: 0.5921 - a_ccc: 0.4614 - d_ccc: 0.4619 - val_loss: 1.9167 - val_v_loss: 0.6472 - val_a_loss: 0.6248 - val_d_loss: 0.6446 - val_v_ccc: 0.3528 - val_a_ccc: 0.3752 - val_d_ccc: 0.3554
Epoch 3/50
6400/6400 [==============================] - 87s 14ms/step - loss: 1.2278 - v_loss: 0.3046 - a_loss: 0.4629 - d_loss: 0.4603 - v_ccc: 0.6954 - a_ccc: 0.5371 - d_ccc: 0.5397 - val_loss: 1.9114 - val_v_loss: 0

Epoch 5/50
6400/6400 [==============================] - 87s 14ms/step - loss: 1.0189 - v_loss: 0.2399 - a_loss: 0.3965 - d_loss: 0.3824 - v_ccc: 0.7601 - a_ccc: 0.6035 - d_ccc: 0.6176 - val_loss: 1.8496 - val_v_loss: 0.5866 - val_a_loss: 0.6079 - val_d_loss: 0.6551 - val_v_ccc: 0.4134 - val_a_ccc: 0.3921 - val_d_ccc: 0.3449
Epoch 6/50
6400/6400 [==============================] - 89s 14ms/step - loss: 0.9248 - v_loss: 0.2132 - a_loss: 0.3656 - d_loss: 0.3460 - v_ccc: 0.7868 - a_ccc: 0.6344 - d_ccc: 0.6540 - val_loss: 1.8661 - val_v_loss: 0.5870 - val_a_loss: 0.6170 - val_d_loss: 0.6621 - val_v_ccc: 0.4130 - val_a_ccc: 0.3830 - val_d_ccc: 0.3379
Epoch 7/50
6400/6400 [==============================] - 91s 14ms/step - loss: 0.8414 - v_loss: 0.1970 - a_loss: 0.3333 - d_loss: 0.3111 - v_ccc: 0.8030 - a_ccc: 0.6667 - d_ccc: 0.6889 - val_loss: 1.8497 - val_v_loss: 0.5593 - val_a_loss: 0.5966 - val_d_loss: 0.6938 - val_v_ccc: 0.4407 - val_a_ccc: 0.4034 - val_d_ccc: 0.3062
Epoch 8/50
6400/6400 [

Epoch 10/50
6400/6400 [==============================] - 89s 14ms/step - loss: 0.6323 - v_loss: 0.1443 - a_loss: 0.2525 - d_loss: 0.2356 - v_ccc: 0.8557 - a_ccc: 0.7475 - d_ccc: 0.7644 - val_loss: 1.8152 - val_v_loss: 0.5589 - val_a_loss: 0.6125 - val_d_loss: 0.6437 - val_v_ccc: 0.4411 - val_a_ccc: 0.3875 - val_d_ccc: 0.3563
Epoch 11/50
6400/6400 [==============================] - 90s 14ms/step - loss: 0.5845 - v_loss: 0.1362 - a_loss: 0.2327 - d_loss: 0.2157 - v_ccc: 0.8638 - a_ccc: 0.7673 - d_ccc: 0.7843 - val_loss: 1.8293 - val_v_loss: 0.5423 - val_a_loss: 0.6328 - val_d_loss: 0.6542 - val_v_ccc: 0.4577 - val_a_ccc: 0.3672 - val_d_ccc: 0.3458
Epoch 12/50
6400/6400 [==============================] - 93s 14ms/step - loss: 0.5347 - v_loss: 0.1266 - a_loss: 0.2138 - d_loss: 0.1944 - v_ccc: 0.8734 - a_ccc: 0.7862 - d_ccc: 0.8056 - val_loss: 1.8842 - val_v_loss: 0.5608 - val_a_loss: 0.6362 - val_d_loss: 0.6871 - val_v_ccc: 0.4392 - val_a_ccc: 0.3638 - val_d_ccc: 0.3129
Epoch 13/50
6400/64

Epoch 17/50
2039/2039 [==============================] - 5s 2ms/step
[0.43657395243644714, 0.4164949357509613, 0.4142543077468872]
3
Train on 6400 samples, validate on 1600 samples
Epoch 1/50
6400/6400 [==============================] - 96s 15ms/step - loss: 1.9822 - v_loss: 0.6618 - a_loss: 0.6603 - d_loss: 0.6600 - v_ccc: 0.3382 - a_ccc: 0.3397 - d_ccc: 0.3400 - val_loss: 2.1920 - val_v_loss: 0.7563 - val_a_loss: 0.6993 - val_d_loss: 0.7364 - val_v_ccc: 0.2437 - val_a_ccc: 0.3007 - val_d_ccc: 0.2636
Epoch 2/50
6400/6400 [==============================] - 90s 14ms/step - loss: 1.4363 - v_loss: 0.3804 - a_loss: 0.5294 - d_loss: 0.5265 - v_ccc: 0.6196 - a_ccc: 0.4706 - d_ccc: 0.4735 - val_loss: 1.9540 - val_v_loss: 0.6565 - val_a_loss: 0.6323 - val_d_loss: 0.6652 - val_v_ccc: 0.3435 - val_a_ccc: 0.3677 - val_d_ccc: 0.3348
Epoch 3/50
6400/6400 [==============================] - 86s 14ms/step - loss: 1.2458 - v_loss: 0.3084 - a_loss: 0.4767 - d_loss: 0.4607 - v_ccc: 0.6916 - a_ccc: 0.5233

Epoch 25/50
6400/6400 [==============================] - 85s 13ms/step - loss: 0.2513 - v_loss: 0.0617 - a_loss: 0.0993 - d_loss: 0.0903 - v_ccc: 0.9383 - a_ccc: 0.9007 - d_ccc: 0.9097 - val_loss: 1.8783 - val_v_loss: 0.5686 - val_a_loss: 0.6539 - val_d_loss: 0.6558 - val_v_ccc: 0.4314 - val_a_ccc: 0.3461 - val_d_ccc: 0.3442
Epoch 26/50
6400/6400 [==============================] - 86s 13ms/step - loss: 0.2379 - v_loss: 0.0578 - a_loss: 0.0932 - d_loss: 0.0869 - v_ccc: 0.9422 - a_ccc: 0.9068 - d_ccc: 0.9131 - val_loss: 1.8965 - val_v_loss: 0.5664 - val_a_loss: 0.6483 - val_d_loss: 0.6819 - val_v_ccc: 0.4336 - val_a_ccc: 0.3517 - val_d_ccc: 0.3181
Epoch 27/50
6400/6400 [==============================] - 86s 13ms/step - loss: 0.2293 - v_loss: 0.0559 - a_loss: 0.0907 - d_loss: 0.0827 - v_ccc: 0.9441 - a_ccc: 0.9093 - d_ccc: 0.9173 - val_loss: 1.8822 - val_v_loss: 0.5635 - val_a_loss: 0.6601 - val_d_loss: 0.6586 - val_v_ccc: 0.4365 - val_a_ccc: 0.3399 - val_d_ccc: 0.3414
Epoch 28/50
6400/64

...

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



6400/6400 [==============================] - 85s 13ms/step - loss: 1.0925 - v_loss: 0.2638 - a_loss: 0.4273 - d_loss: 0.4014 - v_ccc: 0.7362 - a_ccc: 0.5727 - d_ccc: 0.5986 - val_loss: 2.0685 - val_v_loss: 0.6723 - val_a_loss: 0.7151 - val_d_loss: 0.6811 - val_v_ccc: 0.3277 - val_a_ccc: 0.2849 - val_d_ccc: 0.3189
Epoch 5/50
6400/6400 [==============================] - 86s 13ms/step - loss: 0.9684 - v_loss: 0.2311 - a_loss: 0.3768 - d_loss: 0.3606 - v_ccc: 0.7689 - a_ccc: 0.6232 - d_ccc: 0.6394 - val_loss: 1.8056 - val_v_loss: 0.5908 - val_a_loss: 0.6023 - val_d_loss: 0.6125 - val_v_ccc: 0.4092 - val_a_ccc: 0.3977 - val_d_ccc: 0.3875
Epoch 6/50
6400/6400 [==============================] - 85s 13ms/step - loss: 0.8981 - v_loss: 0.2119 - a_loss: 0.3550 - d_loss: 0.3312 - v_ccc: 0.7881 - a_ccc: 0.6450 - d_ccc: 0.6688 - val_loss: 1.8235 - val_v_loss: 0.5847 - val_a_loss: 0.5941 - val_d_loss: 0.6447 - val_v_ccc: 0.4153 - val_a_ccc: 0.4059 - val_d_ccc: 0.3553
Epoch 7/50
6400/6400 [===========

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



6400/6400 [==============================] - 84s 13ms/step - loss: 0.4968 - v_loss: 0.1180 - a_loss: 0.1977 - d_loss: 0.1811 - v_ccc: 0.8820 - a_ccc: 0.8023 - d_ccc: 0.8189 - val_loss: 1.8757 - val_v_loss: 0.5404 - val_a_loss: 0.6378 - val_d_loss: 0.6975 - val_v_ccc: 0.4596 - val_a_ccc: 0.3622 - val_d_ccc: 0.3025
Epoch 14/50
6400/6400 [==============================] - 86s 13ms/step - loss: 0.4582 - v_loss: 0.1100 - a_loss: 0.1805 - d_loss: 0.1678 - v_ccc: 0.8900 - a_ccc: 0.8195 - d_ccc: 0.8322 - val_loss: 1.9379 - val_v_loss: 0.5602 - val_a_loss: 0.6781 - val_d_loss: 0.6996 - val_v_ccc: 0.4398 - val_a_ccc: 0.3219 - val_d_ccc: 0.3004
Epoch 15/50
6400/6400 [==============================] - 86s 13ms/step - loss: 0.4294 - v_loss: 0.1007 - a_loss: 0.1722 - d_loss: 0.1566 - v_ccc: 0.8993 - a_ccc: 0.8278 - d_ccc: 0.8434 - val_loss: 1.8790 - val_v_loss: 0.5506 - val_a_loss: 0.6513 - val_d_loss: 0.6770 - val_v_ccc: 0.4494 - val_a_ccc: 0.3487 - val_d_ccc: 0.3230
Epoch 16/50
6400/6400 [========

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

